In [1]:
import json
import seaborn as sns
import pandas as pd
import os
import fnmatch
import numpy as np
import matplotlib.pyplot as plt
import datetime
import requests
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import csv 


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# data = "/content/gdrive/My Drive/dsf/data"
# root = "/content/gdrive/My Drive/dsf"

In [ ]:
columns = ['created_at', 'description', 'id', 'id_str', 'location', 'favourites_count', 'followers_count',
           'friends_count', 'screen_name', 'statuses_count', 'verified', 'listed_count', 'geo', 'place', 'lang']

In [ ]:
def jstring(filename):
    jsonString= "["
    with open(filename) as fp:
        line = fp.readline()
        while line:
            jsonString += (line+",")
            line = fp.readline()
    jsonString = jsonString[:-1] + "]"
    return json.loads(jsonString)

def getLists(jsonString):
    myjson = jsonString
    delete_list, created_list = [],[]
    for l in jsonString:
        try :
            if l['created_at'] is not None:
                l['user']['geo'] = l['geo']
                l['user']['place'] = l['place']
                l['user']['lang'] = l['lang']
                created_list.append(l['user'])
        except:
            delete_list.append(l)
    return delete_list, created_list

def getpd(filename):
    jsonString = jstring(filename)
    print(len(jsonString))
    delete_list, created_list = getLists(jsonString)
    created_df = pd.DataFrame(created_list)
    created_df = created_df[columns]
    return created_df

def getzippedfile(df1, df2):
    frames = [df1, df2]
    return pd.concat(frames)

def getData(path):
    result = pd.DataFrame()
    for filename in os.listdir(path):
        print(filename)
        if filename.endswith(".txt"):
            df = getpd(data+"/"+filename)
            result = getzippedfile(result, df)
            print(len(result))
    return result

def csv_save(df, file):
    df.to_csv(file, encoding = 'utf-8', index = None, header=True)

def csv_load(file):
    return pd.read_csv(file, encoding = 'utf-8', lineterminator='\n')

def getCitites(csv_file):
    global cities
    cities = []
    with open(csv_file, 'r') as file:
      csv_reader = csv.reader(file)
      # extracting field names through first row 
      fields = next(csv_reader)
      for row in csv_reader:
        cities.append([row[0], row[1]])

def getCityUsingCoordinate(lat, lon):
    try:
        geolocator = Nominatim(user_agent="DSF")
        geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)
        location = geocode(lat + "," + lon)
        if "county" in location.raw["address"]:
            return location.raw["address"]["county"]
        if "city" in location.raw["address"]:
            return location.raw["address"]["city"]
    except:
        return None

def getCityName(x):
    global cities
    x = x.lower()
    for c in cities:
        if c[0].lower() in x or c[1].lower() in x:
            return c[0]

def getCity(x):
    try:
        if x['place'] != "None":
            obj = json.loads(x['place'].replace("'", "\""))
            if obj['place_type'] == 'city':
                return obj['name']
    except:
        pass
    try:
        if x['geo'] != "None":
            obj = json.loads(x['geo'].replace("'", "\""))
            obj = obj['coordinates']
            if obj is not None:
                return getCityUsingCoordinate( str(obj[0]), str(obj[1]) )
    except:
        pass
    try:
        return getCityName(x['location'])
    except:
        pass
    return None